# Linear Model

We assume
    $$y = X \beta + \epsilon$$
where $y$ is a $N \times 1$ vector, $X$ is a $N \times p$ matrix, and $\epsilon$ is a $N \times 1$ vector.

In [1]:
import numpy as np

np.random.seed(0)

N = 100
p = 10
epsilon_stdev = 0.1

X = np.random.rand(N, p)
true_beta = np.random.rand(p, 1)
y = np.matmul(X, true_beta) + epsilon_stdev*np.random.randn(N, 1)
true_beta

array([[0.59288027],
       [0.0100637 ],
       [0.4758262 ],
       [0.70877039],
       [0.04397543],
       [0.87952148],
       [0.52008142],
       [0.03066105],
       [0.22441361],
       [0.9536757 ]])

In [2]:
def estimator_accuracy(estimator):
    print(f"MSE on training data: {estimator.mse(y, X)}")
    print(f"R squared on training data: Unadjusted: {estimator.r_squared(y, X)} Adjusted: {estimator.adjusted_r_squared(y, X)}")
    print(f"Sum of absolute differences of estimator to true estimate {np.sum(np.abs(estimator.get_params() - true_beta))}")

# OLS

## Normal equation

$$\hat{\beta} = (X^TX)^{-1} X^T y$$

In [3]:
import linear_model
ols = linear_model.OLS(standardize=False)
ols.fit_by_closed_form(y, X)
estimator_accuracy(ols)

MSE on training data: 0.007066663029624299
R squared on training data: Unadjusted: 0.9763211602012537 Adjusted: 0.9739532762213792
Sum of absolute differences of estimator to true estimate 0.2353307624755655


## Gradient Descent

Given 
    $$\ssr = ||y - X \beta||_2^2,$$
we update $\beta_n \to \beta_{n+1}$ in the opposite direction of the gradient of $\ssr$ wrt to $\beta$ at $\beta_n$.

We have
    $$d \ssr(\beta) = d (y - X \beta)^T (y - X \beta) = 2(y - X \beta)^T (-X d \beta)$$
    $$\implies D \ssr(\beta) = 2(y - X \beta)^T (-X)$$
    $$\implies \nabla_\beta \ssr = (D \ssr(\beta))^T = -2X^T (y - X \beta).$$
Thus, our update rule is
    $$\beta_{n+1} = \beta_n - \nabla_{\beta} \ssr(\beta_n) = \beta_n + 2X^T(y - X \beta_n).$$

In [4]:
np.random.seed(100)
ols.fit_by_gradient_descent(y, X, iter=10000)
estimator_accuracy(ols)

MSE on training data: 0.012223918848938205
R squared on training data: Unadjusted: 0.9590328670990952 Adjusted: 0.9549361538090047
Sum of absolute differences of estimator to true estimate 0.7824317345507632


## Coordinate Descent

The OLS
    $$\DeclareMathOperator{\col}{\text{col}} \DeclareMathOperator{\row}{\text{row}} \DeclareMathOperator{\ssr}{\text{SSR}} \hat{\beta} = \arg \min \text{SSR} = \arg \min_\beta ||y - X \beta||_2^2 = ||y - \sum_{i=1}^p \col_i X \beta_i||_2^2$$
    
Coordinate descent minimizes the SSR with respect to one coordinate $\beta_j$ at a time:
    $$d \ssr(\beta_j) = (- \col_j X d \beta_j)^T (y - \sum_{i=1}^p \col_i X \beta_i) + (y - \sum_{i=1}^p \col_i X \beta_i)^T (- \col_j X d \beta_j) = -2(y - \sum_{i=1}^p \col_i X \beta_i)^T \col_j X d \beta_j$$
    $$\implies \frac{\partial \ssr}{\partial \beta_j} = -2(y - \sum_{i=1}^p \col_i X \beta_i)^T \col_j X.$$
Setting the partial derivative to 0,
    $$\frac{\partial \ssr}{\partial \beta_j} = 0$$
    $$\begin{align*}
    \implies 0 &= (y - \sum_{i=1}^p \col_i X \beta_i)^T \col_j X \\
               &= (y - \sum_{i \neq j}^p \col_i X \beta_i - \col_j X \beta_j)^T \col_j X \\
               &= (y - \sum_{i \neq j}^p \col_i X \beta_i)^T \col_j X - (\col_j X \beta_j)^T \col_j X \\
               &= (y - \sum_{i \neq j}^p \col_i X \beta_i)^T \col_j X - \beta_j ||col_j X||_2^2 \\
    \end{align*}$$
    $$\begin{align*}
    \implies \beta_j &= \frac{1}{||\col_j X||_2^2} (y - \sum_{i \neq j}^p \col_i X \beta_i)^T \col_j X \\
    &= \frac{1}{||\col_j X||_2^2} (y - X \beta + \col_j X \beta_j)^T \col_j X 
    \end{align*}$$

In [5]:
np.random.seed(100)
ols.fit_by_coordinate_descent(y, X, iter=10000)
estimator_accuracy(ols)

MSE on training data: 0.0070666630296242925
R squared on training data: Unadjusted: 0.9763211602012538 Adjusted: 0.9739532762213792
Sum of absolute differences of estimator to true estimate 0.2353307624755792


# Ridge

## Closed Form Equation

$$\hat{\beta}_{\lambda}^{\text{ridge}} = (X^T X + \lambda I_p)^{-1} X^T y$$

To be efficient, we should SVD $X$ to get a faster inverse of the $X^TX$ term.

In [6]:
ridge = linear_model.Ridge(standardize=False)
ridge.fit_by_closed_form(y, X, reg_param=0.5)
estimator_accuracy(ridge)

MSE on training data: 0.007389897382654656
R squared on training data: Unadjusted: 0.9752726915122735 Adjusted: 0.9727999606635007
Sum of absolute differences of estimator to true estimate 0.3136167340163366


## Gradient descent

Given 
    $$\ssr = ||y - X \beta||_2^2 + \lambda ||\beta||_2^2,$$
we update $\beta_n \to \beta_{n+1}$ in the opposite direction of the gradient of $\ssr$ wrt to $\beta$ at $\beta_n$.

We have
    $$d \ssr(\beta) = d (y - X \beta)^T (y - X \beta) + d \lambda \beta^T \beta.$$
The differential of the regularization term is
    $$d \beta^T \beta = (d \beta)^T \beta + \beta^T d \beta = 2 \lambda \beta^T d \beta,$$
which, combined with the differential of cost term calculated above, implies
    $$\nabla_\beta \ssr = (D \ssr(\beta))^T = -2X^T (y - X \beta) + 2 \lambda \beta.$$
Thus, our update rule is
    $$\beta_{n+1} = \beta_n - \nabla_{\beta} \ssr(\beta_n) = \beta_n + 2X^T(y - X \beta_n) - 2 \lambda \beta.$$

In [7]:
ridge = linear_model.Ridge(standardize=False)
np.random.seed(100)
ridge.fit_by_gradient_descent(y, X, reg_param=0.5)
estimator_accuracy(ridge)

MSE on training data: 0.007389893854692402
R squared on training data: Unadjusted: 0.9752727035644682 Adjusted: 0.972799973920915
Sum of absolute differences of estimator to true estimate 0.313618359654561


# Lasso

## Coordinate descent

The lasso
    $$\hat{\beta}_\lambda^{\text{lasso}} = \arg \min_\beta \ssr$$
where
    $$\ssr = \frac{1}{2N} ||y - X \beta||_2^2 + \lambda ||\beta||_1$$
Since we have already calculated the update to $\hat{\beta}_j$ for $||y - X \beta||_2^2$, we will focus on the $L_1$ regularization term here:
    $$\frac{\partial}{\partial \beta_j} \lambda ||\beta||_1 = \frac{\partial}{\partial \beta_j} \lambda \sum_{i=1}^p |\beta_j| = \begin{cases} \lambda & \beta_j > 0 \\ -\lambda & \beta_j < 0 \\ \text{undef} & \beta_j = 0 \end{cases} = g(\beta_j),$$
where we let 
    $$g(\beta_j) = \begin{cases} \lambda & \beta_j > 0 \\ -\lambda & \beta_j < 0 \\ \text{undef} & \beta_j = 0 \end{cases}.$$

The subdifferential of the L1 regularization term is
    $$g(\beta_j) = \begin{cases} \lambda & \beta_j > 0 \\ -\lambda & \beta_j < 0 \\ [-\lambda, \lambda] & \beta_j = 0 \end{cases}$$
where the change is having the derivative at $\beta_j = 0$ be a set, instead of being undefined.

Using the derivation for the partial derivative of $||y - X \beta||_2^2$ above, we have
    $$\frac{\partial}{\partial \beta_j} \ssr = -\frac{1}{N} (y - \sum_{i=1}^p \col_i X \beta_i)^T \col_j X + g(\beta_j)$$
    
Similar to the above situation for OLS, we set the partial derivative to 0,
    $$\frac{\partial \ssr}{\partial \beta_j} = 0$$
    $$\begin{align*}
    \implies 0 &= -\frac{1}{N} (y - \sum_{i=1}^p \col_i X \beta_i)^T \col_j X + g(\beta_j) \\
               &\vdots \\
               &= -\frac{1}{N} (y - \sum_{i \neq j}^p \col_i X \beta_i)^T \col_j X + \beta_j ||col_j X||_2^2 + g(\beta_j) \\
    \end{align*}$$
    $$\begin{align*}
    \implies \beta_j &= \frac{1}{||\col_j X||_2^2} (\frac{1}{N} (y - \sum_{i \neq j}^p \col_i X \beta_i)^T \col_j X - g(\beta_j)) \\
    &= \frac{1}{||\col_j X||_2^2} ((y - X \beta + \col_j X \beta_j)^T \col_j X - g(\beta_j))
    \end{align*}$$

### Subdifferential cases
Now we have to go case by case for the subdifferential. Note that we use $\beta_j'$ to indicate the new/updated $\beta_j$.

If $\beta_j > 0$, then
    $$\frac{1}{||\col_j X||_2^2} (\frac{1}{N}(y - X \beta + \col_j X \beta_j)^T \col_j X - g(\beta_j)) > 0$$
    $$\implies \frac{1}{N} (y - X \beta + \col_j X \beta_j)^T \col_j X > g(\beta_j) = \lambda$$
and
    $$\implies \beta_j' = \frac{1}{||\col_j X||_2^2} (\frac{1}{N} (y - X \beta + \col_j X \beta_j)^T \col_j X - \lambda)$$

If $\beta_j < 0$, then
    $$\frac{1}{||\col_j X||_2^2} (\frac{1}{N} (y - X \beta + \col_j X \beta_j)^T \col_j X - g(\beta_j)) < 0$$
    $$\implies \frac{1}{N} (y - X \beta + \col_j X \beta_j)^T \col_j X < g(\beta_j) = -\lambda$$
and
    $$\implies \beta_j' = \frac{1}{||\col_j X||_2^2} (\frac{1}{N} (y - X \beta + \col_j X \beta_j)^T \col_j X + \lambda)$$

If $\beta_j = 0$, then
    $$\frac{1}{||\col_j X||_2^2} (\frac{1}{N} (y - X \beta + \col_j X \beta_j)^T \col_j X - g(\beta_j)) = 0$$
    $$\implies -\lambda \leq \frac{1}{N} (y - X \beta + \col_j X \beta_j)^T \col_j X \leq \lambda$$
and
    $$\implies \beta_j' = 0$$

Summarized, we have the update rule
    $$\beta_j = \begin{cases}
    \frac{1}{||\col_j X||_2^2} (\frac{1}{N} (y - X \beta + \col_j X \beta_j)^T \col_j X - \lambda) & \frac{1}{N} (y - X \beta + \col_j X \beta_j)^T \col_j X > \lambda \\ 
    \frac{1}{||\col_j X||_2^2} (\frac{1}{N} (y - X \beta + \col_j X \beta_j)^T \col_j X + \lambda) & \frac{1}{N} (y - X \beta + \col_j X \beta_j)^T \col_j X < -\lambda \\ 
    0 & -\lambda \leq \frac{1}{N} (y - X \beta + \col_j X \beta_j)^T \col_j X \leq \lambda
    \end{cases}$$
Wwe can rewrite the above using the soft-thresholding function $S()$:
    $$\beta_j = \frac{1}{||\col_j X||_2^2} S(\frac{1}{N} (y - X \beta + \col_j X \beta_j)^T \col_j X, \lambda),$$
where the soft-thresholding function is defined to be
    $$S(a, b) = \begin{cases} a - b & a > b \\ a + b & a < -b \\ 0 & -b \leq a \leq b \end{cases}$$

In [8]:
lasso = linear_model.Lasso(standardize=False)
np.random.seed(100)
lasso.fit_by_coordinate_descent(y, X, reg_param=0.6)
estimator_accuracy(lasso)

MSE on training data: 0.007225805709543815
R squared on training data: Unadjusted: 0.9761336733706564 Adjusted: 0.973747040707722
Sum of absolute differences of estimator to true estimate 0.24739624907662028


## Leave-One-Out Cross Validation for Lasso

1. Choose a grid of P possible values $\lambda_1, \dots, \lambda_P$ for the penalty parameter
2. For $i = 1, \dots, N$, exclude the $i^{\text{th}}$ observation $(y_i, x_i)$ from the sample
    1. Use the remaining $N - 1$ observations to compute $P$ ridge estimates $\hat{\beta}_{\lambda_p, x_i}$ where the subscript $\lambda_p, x_i$ indicates the chosen penalty parameter is $\lambda_p$ where $p = 1, \dots, P$ and the $i^{\text{th}}$ observation has been excluded
    2. Compute $P$ out-of-sample predictions of the excluded observation $\hat{y}_{\lambda_p, x_i} = x_i \hat{\beta}_{\lambda_p, x_i}$ for $p = 1, \dots, P$.
3. Compute the MSE of the predictions 
    $$\text{MSE}_{\lambda_p} = \frac{1}{N} \sum_{i=1}^N (y_i - \hat{y}_{\lambda_p, x_i})^2$$
for $p = 1, \dots, P.$
4. Choose $\lambda^* = \arg \min_{\lambda_p} \text{MSE}_{\lambda_p}$ as your optimal $\lambda$.

### Choosing a maximum value for lasso $\lambda$ grid

The update rule
    $$\beta_j = \begin{cases}
    \frac{1}{||\col_j X||_2^2} ((y - X \beta + \col_j X \beta_j)^T \col_j X - \lambda) & (y - X \beta + \col_j X \beta_j)^T \col_j X > \lambda \\ 
    \frac{1}{||\col_j X||_2^2} ((y - X \beta + \col_j X \beta_j)^T \col_j X + \lambda) & (y - X \beta + \col_j X \beta_j)^T \col_j X < -\lambda \\ 
    0 & -\lambda \leq (y - X \beta + \col_j X \beta_j)^T \col_j X \leq \lambda
    \end{cases}$$
implies $\beta_j$ stays at $0$ if
    $$-\lambda \leq (y - X \beta + \col_j X \beta_j)^T \col_j X \leq \lambda \implies |(y - X \beta + \col_j X \beta_j)^T \col_j X| \leq \lambda$$

Since
    $$|(y - X \beta + \col_j X \beta_j)^T \col_j X| \leq |y^T \col_j X|,$$
then 
    $$|y^T \col_j X| \leq \lambda \implies |(y - X \beta + \col_j X \beta_j)^T \col_j X| \leq \lambda.$$
Then a suitable maximum value for the lasso $\lambda$ grid is
    $$\lambda_{\text{max}} = \max_{j} |y^T \col_j X|$$

In [9]:
%%time
lasso.choose_reg_param_for_lasso(y, X, grid_size=30)

Regularization parameter chosen among 30 evenly spaced values from 0 to 4.426918682517629
CPU times: user 6.88 s, sys: 60 ms, total: 6.94 s
Wall time: 7.02 s


0.45795710508803056